In [1]:
import os

In [2]:
%pwd

'c:\\Users\\viveksharma\\OneDrive\\Desktop\\mlops\\Kideny-Disease-Classification_PROJECT\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\viveksharma\\OneDrive\\Desktop\\mlops\\Kideny-Disease-Classification_PROJECT'

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
import sys
from pathlib import Path

# Current directory is research folder
# We need to go to the project root (Kideny-Disease-Classification_PROJECT)
project_root = Path.cwd()  # This is already the project root after os.chdir("../")
src_path = project_root / 'src'

print(f"Project root: {project_root}")
print(f"SRC path: {src_path}")
print(f"SRC exists: {src_path.exists()}")

# Add src directory to path
sys.path.insert(0, str(src_path))
print(f"Added to sys.path: {sys.path[0]}")

Project root: c:\Users\viveksharma\OneDrive\Desktop\mlops\Kideny-Disease-Classification_PROJECT
SRC path: c:\Users\viveksharma\OneDrive\Desktop\mlops\Kideny-Disease-Classification_PROJECT\src
SRC exists: True
Added to sys.path: c:\Users\viveksharma\OneDrive\Desktop\mlops\Kideny-Disease-Classification_PROJECT\src


In [25]:
# Now the path is already added, we can import
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [30]:
import os
import zipfile
import gdown
from CNN_Classifier import logger
from CNN_Classifier.utils.common import get_size

In [31]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-21 01:15:38,806: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-21 01:15:38,815: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-21 01:15:38,817: INFO: common: created directory at: artifacts]
[2026-01-21 01:15:38,818: INFO: common: created directory at: artifacts/data_ingestion]
[2026-01-21 01:15:38,819: INFO: 986331348: Downloading data from https://drive.google.com/file/d/19NiXTXgVAlrlWZTA63-VVMbL9JP6E9cI/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=19NiXTXgVAlrlWZTA63-VVMbL9JP6E9cI
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=19NiXTXgVAlrlWZTA63-VVMbL9JP6E9cI&confirm=t&uuid=052de714-ea92-4f03-a29b-a357e294db3e
To: c:\Users\viveksharma\OneDrive\Desktop\mlops\Kideny-Disease-Classification_PROJECT\artifacts\data_ingestion\data.zip
100%|██████████| 1.63G/1.63G [05:03<00:00, 5.36MB/s]

[2026-01-21 01:20:46,164: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/19NiXTXgVAlrlWZTA63-VVMbL9JP6E9cI/view?usp=sharing into file artifacts/data_ingestion/data.zip]


: 